In [1]:
# import necessary libraries

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
#matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

# BigQuery settings
from google.cloud import bigquery
from google.cloud.bigquery import dbapi;
client = bigquery.Client("som-nero-phi-jonc101"); # Project identifier
conn = dbapi.connect(client);
cursor = conn.cursor();

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [ ]:
# ignore if local tmp.csv already exists
# read the time matching CSV and remove MRNs for security reasons (ignore if local tmp.csv already exists)
import pandas as pd
data_frame = pd.read_csv('jon_mapping.csv')
data_frame = data_frame.drop('MRN', axis = 1) 
data_frame.to_csv('tmp.csv', index = False)

In [ ]:
# only for testing
# Generate a random mapping (CSV file) to test (only for testing)

import pandas as pd
data_frame = pd.read_csv('tmp.csv')
print(list(data_frame.columns)) # print the column names

num_row = data_frame.shape[0]
print(num_row) # print num_row

data_frame = data_frame.drop('JITTER', axis = 1) # remove a column

data_frame['JITTER_test']= np.random.randint(10, size=num_row) # add a column 

print(data_frame[0:10]) # print the first 10 rows

data_frame.to_csv('tmp_rnd_shift.csv', index = False) # save to CSV

In [ ]:
# Skip if table ADT_cohort_jit already exists on GCP
# Create a table of ADT of cohort patients (with jittered times) (ADT_cohort_jit): 
###.  *** time_out is either the actual time_out or TPA_admin_time, whichever is earlier

query =  """
drop table if exists noshad.ADT_cohort_jit;
create table noshad.ADT_cohort_jit as
(
SELECT ADT.jc_uid, ADT.pat_enc_csn_id_coded, ADT.department_id, CH.tpaAdminTime,
    min(ADT.event_time_jittered) AS time_in, max(ADT.event_time_jittered) AS time_out
FROM `starr_datalake2018.adt` AS ADT
INNER JOIN `noshad_test.cohort_AL_user_role` AS CH
  USING (pat_enc_csn_id_coded)
GROUP BY ADT.jc_uid, ADT.pat_enc_csn_id_coded, ADT.department_id, CH.tpaAdminTime
ORDER BY ADT.jc_uid, ADT.pat_enc_csn_id_coded, time_in
)
"""
cursor.execute(query);

#results = cursor.fetchall();
#print(results[:2])
#results_np = np.array(results)

In [ ]:
## main part to generate the transaction rate associated with other patients per each cohort_patient
# CREATE OR REPLACE TABLE noshad.num_tnx


client = bigquery.Client("som-nero-phi-jonc101"); # Project identifier
conn = dbapi.connect(client);
cursor = conn.cursor();

# Upload time_mapping tmp.CSV 

schemafield_col1 = bigquery.schema.SchemaField("ANON_ID","STRING") #Define your schema
schemafield_col2 = bigquery.schema.SchemaField("JITTER","INTEGER")

filename = 'tmp.csv'
table_id = 'tmp' # the name of the chart to create

dataset_ref = client.dataset('noshad')
table_ref = dataset_ref.table(table_id)

job_config = bigquery.LoadJobConfig()
job_config.source_format = bigquery.SourceFormat.CSV
job_config.skip_leading_rows = 1
job_config.autodetect = True

with open(filename, "rb") as source_file:
    job = client.load_table_from_file(source_file, table_ref, job_config=job_config)

job.result()  # Waits for table load to complete.

print('temporary tmp file generated')

# Main part to generate num_pat per each patient
query= """

CREATE OR REPLACE TABLE noshad.num_tnx AS(
WITH

    -- Generate ADT_cohort with actual times
  ADT_real_date AS
  (SELECT ADT.* except(time_in,time_out,tpaAdminTime), DATETIME_SUB(ADT.time_in, INTERVAL TMP.JITTER DAY) as time_in,  
  DATETIME_SUB(ADT.time_out, INTERVAL TMP.JITTER DAY) as time_out, 
  DATETIME_SUB(ADT.tpaAdminTime, INTERVAL TMP.JITTER DAY) as tpaAdminTime
  
  FROM `noshad.ADT_cohort_jit` as ADT,
  `noshad.tmp` as TMP
  
  WHERE ADT.jc_uid=TMP.ANON_ID
  
  ORDER BY ADT.jc_uid, ADT.pat_enc_csn_id_coded, time_in
  ),
  
    -- Generate AL with actual times
  AL_real_date AS
  (SELECT AL.*, DATETIME_SUB(AL.access_time_jittered, INTERVAL TMP.JITTER DAY) as access_time_real
  FROM `noshad.shc_access_log_de_dep_id` as AL,
  `noshad.tmp` as TMP
  WHERE AL.rit_uid=TMP.ANON_ID
  ORDER BY AL.rit_uid
  ),
  
  --- Generate NUM of PAT PER DEP with times
  NUM_PAT_PER_DEP AS 
  (SELECT ADT_real_date.*, count(*) as num_tranx , 
    count(*)/ DATETIME_DIFF(ADT_real_date.time_out, ADT_real_date.time_in, MINUTE) as num_tranx_rate 
  FROM ADT_real_date, AL_real_date
  WHERE 
    AL_real_date.department_id = ADT_real_date.department_id
    AND ADT_real_date.time_in < AL_real_date.access_time_real 
    AND AL_real_date.access_time_real < ADT_real_date.time_out
    
  GROUP BY ADT_real_date.jc_uid, ADT_real_date.pat_enc_csn_id_coded, 
    ADT_real_date.department_id, ADT_real_date.tpaAdminTime, 
    ADT_real_date.time_in, ADT_real_date.time_out
  ORDER BY ADT_real_date.department_id)

  -- Main script
SELECT jc_uid, pat_enc_csn_id_coded, max(num_tranx_rate) as max_norm_num_tranx
FROM NUM_PAT_PER_DEP
WHERE time_in < tpaAdminTime
GROUP BY jc_uid, pat_enc_csn_id_coded
)"""

cursor.execute(query);

print('feature extraced')

## Final step: delete the temporary time mapping
query = "DROP TABLE noshad.tmp"
cursor.execute(query);

print('temporary tmp file deleted')

In [7]:
## Number of unique patients per each cohort_patient in ED
# CREATE OR REPLACE TABLE noshad.num_unq_pat


client = bigquery.Client("som-nero-phi-jonc101"); # Project identifier
conn = dbapi.connect(client);
cursor = conn.cursor();

# Upload time_mapping tmp.CSV 

schemafield_col1 = bigquery.schema.SchemaField("ANON_ID","STRING") #Define your schema
schemafield_col2 = bigquery.schema.SchemaField("JITTER","INTEGER")

filename = 'tmp.csv'
table_id = 'tmp' # the name of the chart to create

dataset_ref = client.dataset('noshad')
table_ref = dataset_ref.table(table_id)

job_config = bigquery.LoadJobConfig()
job_config.source_format = bigquery.SourceFormat.CSV
job_config.skip_leading_rows = 1
job_config.autodetect = True

with open(filename, "rb") as source_file:
    job = client.load_table_from_file(source_file, table_ref, job_config=job_config)

job.result()  # Waits for table load to complete.

print('temporary tmp file generated')

# Main part to generate num_pat per each patient
query= """

CREATE OR REPLACE TABLE noshad.num_unq_pat AS(
WITH

    -- Generate ADT_cohort with actual times
  ADT_real_date AS
  (SELECT ADT.* except(time_in,time_out,tpaAdminTime), DATETIME_SUB(ADT.time_in, INTERVAL TMP.JITTER DAY) as time_in,  
  DATETIME_SUB(ADT.time_out, INTERVAL TMP.JITTER DAY) as time_out, 
  DATETIME_SUB(ADT.tpaAdminTime, INTERVAL TMP.JITTER DAY) as tpaAdminTime
  
  FROM `noshad.ADT_cohort_jit` as ADT,
  `noshad.tmp` as TMP
  
  WHERE ADT.jc_uid=TMP.ANON_ID
  
  ORDER BY ADT.jc_uid, ADT.pat_enc_csn_id_coded, time_in
  ),
  
    -- Generate AL with actual times
  AL_real_date AS
  (SELECT AL.*, DATETIME_SUB(AL.access_time_jittered, INTERVAL TMP.JITTER DAY) as access_time_real
  FROM `noshad.shc_access_log_de_dep_id` as AL,
  `noshad.tmp` as TMP
  WHERE AL.rit_uid=TMP.ANON_ID
  ORDER BY AL.rit_uid
  ),
  
  --- Generate NUM of Unique PAT PER DEP with times
  NUM_PAT_PER_DEP AS 
  (SELECT ADT_real_date.*, count(DISTINCT AL_real_date.rit_uid) as num_unq_pat , 
    count(DISTINCT AL_real_date.rit_uid)/ DATETIME_DIFF(ADT_real_date.time_out, ADT_real_date.time_in, MINUTE) as num_unq_pat_rate 
  FROM ADT_real_date, AL_real_date
  WHERE 
    AL_real_date.department_id = ADT_real_date.department_id
    AND ADT_real_date.time_in < AL_real_date.access_time_real 
    AND AL_real_date.access_time_real < ADT_real_date.time_out
    
  GROUP BY ADT_real_date.jc_uid, ADT_real_date.pat_enc_csn_id_coded, 
    ADT_real_date.department_id, ADT_real_date.tpaAdminTime, 
    ADT_real_date.time_in, ADT_real_date.time_out
  ORDER BY ADT_real_date.department_id)

  -- Main script
SELECT jc_uid, pat_enc_csn_id_coded, min(num_unq_pat) as max_num_unq_pat
FROM NUM_PAT_PER_DEP
WHERE time_in < tpaAdminTime
GROUP BY jc_uid, pat_enc_csn_id_coded
)"""

cursor.execute(query);

print('feature extraced')

## Final step: delete the temporary time mapping
query = "DROP TABLE noshad.tmp"
cursor.execute(query);

print('temporary tmp file deleted')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


temporary tmp file generated
feature extraced
temporary tmp file deleted


In [4]:
## main part to generate the number of new incoming patients (admitted or transferred in) per each cohort_patient
# CREATE OR REPLACE TABLE noshad.num_new_pat


client = bigquery.Client("som-nero-phi-jonc101"); # Project identifier
conn = dbapi.connect(client);
cursor = conn.cursor();

# Upload time_mapping tmp.CSV 

schemafield_col1 = bigquery.schema.SchemaField("ANON_ID","STRING") #Define your schema
schemafield_col2 = bigquery.schema.SchemaField("JITTER","INTEGER")

filename = 'tmp.csv'
table_id = 'tmp' # the name of the chart to create

dataset_ref = client.dataset('noshad')
table_ref = dataset_ref.table(table_id)

job_config = bigquery.LoadJobConfig()
job_config.source_format = bigquery.SourceFormat.CSV
job_config.skip_leading_rows = 1
job_config.autodetect = True

with open(filename, "rb") as source_file:
    job = client.load_table_from_file(source_file, table_ref, job_config=job_config)

job.result()  # Waits for table load to complete.

print('temporary tmp file generated')

# Main part to generate num_pat per each patient
query= """

CREATE OR REPLACE TABLE noshad.num_new_pat AS(
WITH

    -- Generate ADT_cohort with actual times
  ADT_real_date AS
  (SELECT ADT.* except(time_in,time_out,tpaAdminTime), DATETIME_SUB(ADT.time_in, INTERVAL TMP.JITTER DAY) as time_in,  
  DATETIME_SUB(ADT.time_out, INTERVAL TMP.JITTER DAY) as time_out, 
  DATETIME_SUB(ADT.tpaAdminTime, INTERVAL TMP.JITTER DAY) as tpaAdminTime
  
  FROM `noshad.ADT_cohort_jit` as ADT,
  `noshad.tmp` as TMP
  
  WHERE ADT.jc_uid=TMP.ANON_ID
  
  ORDER BY ADT.jc_uid, ADT.pat_enc_csn_id_coded, time_in
  ),
  
  
      -- Generate ADT_summary (pertaining to all patients) with actual times
  ADT_all_real_date AS
  (SELECT ADT.* except(in_time,out_time), DATETIME_SUB(ADT.in_time, INTERVAL TMP.JITTER DAY) as time_in,  
  DATETIME_SUB(ADT.out_time, INTERVAL TMP.JITTER DAY) as time_out
  
  FROM `noshad.adt_summary` as ADT,
  `noshad.tmp` as TMP
  
  WHERE ADT.jc_uid=TMP.ANON_ID
  
  ORDER BY ADT.jc_uid, ADT.pat_enc_csn_id_coded, time_in
  ),
  
  
  --- Generate NUM of New PAT PER DEP with times
  NUM_PAT_PER_DEP AS 
  (SELECT ADT_real_date.*, count(*) as num_new_pat , 
    count(*)/ DATETIME_DIFF(ADT_real_date.time_out, ADT_real_date.time_in, MINUTE) as num_new_pat_rate 
  FROM ADT_real_date, ADT_all_real_date
  WHERE 
    ADT_all_real_date.department_id = ADT_real_date.department_id
    AND ADT_real_date.time_in < ADT_all_real_date.time_in
    AND ADT_all_real_date.time_in < ADT_real_date.time_out
    
  GROUP BY ADT_real_date.jc_uid, ADT_real_date.pat_enc_csn_id_coded, 
    ADT_real_date.department_id, ADT_real_date.tpaAdminTime, 
    ADT_real_date.time_in, ADT_real_date.time_out
  ORDER BY ADT_real_date.department_id)

  -- Main script
SELECT jc_uid, pat_enc_csn_id_coded, max(num_new_pat_rate) as max_num_new_pat_rate,  sum(num_new_pat_rate) as sum_num_new_pat_rate
FROM NUM_PAT_PER_DEP
WHERE time_in < tpaAdminTime
GROUP BY jc_uid, pat_enc_csn_id_coded
)"""

cursor.execute(query);

print('feature extraced')

## Final step: delete the temporary time mapping
query = "DROP TABLE noshad.tmp"
cursor.execute(query);

print('temporary tmp file deleted')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


temporary tmp file generated
feature extraced
temporary tmp file deleted
